In [1]:
import pandas as pd
import gzip
import json
from surprise import BaselineOnly, SVD, Dataset, Reader, accuracy
from sklearn.model_selection import train_test_split

In [2]:
business_file = "final_business_CA.gzip"
reviews_file = "final_review_CA.gzip"
users_file = "final_data_user_yelp.gzip"

In [3]:
### Do data preprocessing containing dictionary of user, businesses and star ratings
bussiness_df = pd.read_pickle(business_file)
reviews_df = pd.read_pickle(reviews_file)

In [29]:
busines = bussiness_df[bussiness_df['state'] == "CA"]['business_id'].tolist()

In [31]:
type(busines)

list

In [33]:
reviews_df[reviews_df['business_id'].isin(busines)]

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
9,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06
23,eCiWBf1CJ0Zdv1uVarEhhw,OhECKhQEexFypOMY6kypRw,vC2qm1y3Au5czBtbhc-DNw,4,0,0,0,"Yes, this is the only sushi place in town. How...",2013-09-04 03:48:20
31,YbMyvlDA2W3Py5lTz8VK-A,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,5,0,0,0,"Great burgers,fries and salad! Burgers have a...",2017-01-02 03:17:34
35,L0jv8c2FbpWSlfNC6bbUEA,bFPdtzu11Oi0f92EAcjqmg,IDtLPgUrqorrpqSLdfMhZQ,5,0,0,0,What a great addition to the Funk Zone! Grab ...,2016-10-13 22:50:47
61,4zopEEPqfwm-c_FNpeHZYw,JYYYKt6TdVA4ng9lLcXt_g,SZU9c8V2GuREDN5KgyHFJw,5,0,0,0,We were a bit weary about trying the Shellfish...,2016-05-31 02:14:54
...,...,...,...,...,...,...,...,...,...
6990140,aGiUeKfrQx5tNLOpPYo7QQ,_n-J_FR8DtWvsP1v2A6lAw,U1Gc3wKN5viu_e68KJVOdQ,1,0,0,0,Sorry... how did person say that it was good? ...,2021-11-11 03:33:31
6990155,UKSGqYVh1g5opUOM4FLIlQ,ut4UutJ_wDVkSaJtf9Ic3A,iT78kOB6Ja3BJQdYipPa5g,5,0,0,0,Amazing dishes & great customer service! This ...,2021-11-24 03:49:21
6990167,4BknKk03Mxwqp6YoZeu1sA,26_8S7BIL6YRw2PcS99Tnw,fiQxkb1pfPpvrr0oYYYBcA,5,0,0,0,"If I could give more than 5 stars, I would! I ...",2021-12-06 22:49:57
6990245,_USUb1lBWnO4VV58FHL6fw,ik0UUieUX38rPg24heeVgw,6aUb_f9C7a2oOxaIfRGSdA,4,0,0,1,I ordered a half rack of Pork Ribs (a la carte...,2019-04-29 18:10:03


In [9]:
max_word_length = lambda x: len(x.split())
tmp['Word_Length'] = tmp['text'].apply(max_word_length)

/var/folders/rd/1wqnsccd7yv7906kcv9tdzbc0000gn/T/ipykernel_45522/1398308717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Word_Length'] = tmp['text'].apply(max_word_length)


In [22]:
len(tmp.iloc[0].text.split())

101

In [4]:
reviews_df = reviews_df[['user_id', 'business_id', 'stars']]

In [5]:
train_df, test_df = train_test_split(reviews_df, train_size=0.8)

In [7]:
valid_df, test_df = train_test_split(test_df, train_size=0.5)

In [8]:
reader = Reader(rating_scale=(0.0, 5.0))
train_set = Dataset.load_from_df(train_df, reader).build_full_trainset()
val_set = Dataset.load_from_df(valid_df, reader).build_full_trainset().build_testset()
test_set = Dataset.load_from_df(test_df, reader).build_full_trainset().build_testset()

In [18]:
bsl_options = {"method": "als", "n_epochs": 30, 'reg_u':0.005,'reg_i':0.005}
bias_only_model = BaselineOnly(bsl_options=bsl_options)

In [19]:
bias_only_model.fit(train_set)

Estimating biases using als...


In [10]:
predictions = bias_only_model.test(val_set)
accuracy.mse(predictions)

NameError: name 'bias_only_model' is not defined

In [9]:
### For SVD Factorization models
bias_sgd = SVD(n_factors=50, n_epochs=30, lr_all=0.005, reg_all=0.005)
bias_sgd.fit(train_set)

In [11]:
predictions = bias_sgd.test(val_set)
accuracy.mse(predictions)

MSE: 1.6234


1.6234163907377122

In [24]:
predictions = bias_sgd.test(test_set)
accuracy.mse(predictions)

MSE: 1.6258


1.6257967707165162

In [12]:
predictions

[Prediction(uid='dTiuUeT2c7-JcA_ta3TC7g', iid='GgcvRnt5_z3NEC0D6vNncQ', r_ui=3.0, est=3.5917144798665745, details={'was_impossible': False}),
 Prediction(uid='HfvmPzXxjrffNeloMS5juw', iid='EtM079Cj7-B3G7jPsGYb_Q', r_ui=5.0, est=3.7484690404121053, details={'was_impossible': False}),
 Prediction(uid='FQPkv0r1D0S2hYrQriRRHA', iid='U3grYFIeu6RgAAQgdriHww', r_ui=2.0, est=4.05916712343898, details={'was_impossible': False}),
 Prediction(uid='GYWLmLU7_EC9lJd2XPAjNg', iid='RjOnk_W82F2brYtpDZ59MA', r_ui=5.0, est=4.179861679213505, details={'was_impossible': False}),
 Prediction(uid='kTqWyzGZ7OvNy3KxqWw1iw', iid='U3grYFIeu6RgAAQgdriHww', r_ui=5.0, est=4.463330990691828, details={'was_impossible': False}),
 Prediction(uid='kTqWyzGZ7OvNy3KxqWw1iw', iid='ycbmvmBYk4w_ZmdrPL0tBQ', r_ui=5.0, est=4.011938949961242, details={'was_impossible': False}),
 Prediction(uid='LIuxlPO-K2rwVQj_X9T-fw', iid='BttSAciQNIQhgJCv3-FYpw', r_ui=5.0, est=4.172721436698179, details={'was_impossible': False}),
 Prediction(

In [34]:
business_CA_list = bussiness_df[bussiness_df['state'] == "CA"]['business_id'].tolist()

In [35]:
business_CA = bussiness_df[bussiness_df['state'] == "CA"]
reviews_CA = reviews_df[reviews_df['business_id'].isin(business_CA_list)]

In [36]:
reviews_CA.to_pickle("final_review_CA.gzip")

In [37]:
business_CA.to_pickle("final_business_CA.gzip")

In [13]:
reviews_df.head(4)

,user_id,business_id,stars
9,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3
23,OhECKhQEexFypOMY6kypRw,vC2qm1y3Au5czBtbhc-DNw,4
31,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,5
35,bFPdtzu11Oi0f92EAcjqmg,IDtLPgUrqorrpqSLdfMhZQ,5


In [14]:
for i, df in reviews_df[:10].iterrows():
    s = df['stars']
    print(type(s), s)

<class 'int'> 3
<class 'int'> 4
<class 'int'> 5
<class 'int'> 5
<class 'int'> 5
<class 'int'> 1
<class 'int'> 1
<class 'int'> 4
<class 'int'> 1
<class 'int'> 5


In [18]:
users_review_df = reviews_df.groupby('user_id')[['business_id']].agg(list).reset_index()

In [21]:
len(users_review_df)

106529

In [22]:
bias_sgd.bu

array([ 0.12583002,  0.41011308, -0.03385642, ...,  0.12107365,
       -0.18347624,  0.05056835])

In [25]:
bias_sgd.pu[0]

array([ 0.14737106, -0.02989309, -0.04298456, -0.02426513, -0.04925075,
        0.1120388 ,  0.02713183, -0.02511422,  0.04583575,  0.01718425,
        0.19067594, -0.05859581,  0.02063064, -0.0509811 , -0.02906613,
        0.0640663 , -0.11294871,  0.10790617,  0.0769577 , -0.00331608,
        0.11200439, -0.0864433 ,  0.02289804, -0.0041401 ,  0.25345775,
        0.09346108,  0.10018229,  0.03928605,  0.07992191,  0.03082013,
       -0.06299093,  0.00339385, -0.12793513, -0.14733703,  0.06191442,
       -0.01496087,  0.08239569, -0.12132292, -0.04019213,  0.03972109,
       -0.21556226,  0.00956667,  0.09736317,  0.0693087 ,  0.10857891,
       -0.05936312,  0.20168542, -0.09229381, -0.0881294 , -0.0058648 ])